In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v10.csv"
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v11.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_CHAOS_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_LVSC_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_2_20_2024_merged_v13.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_16_2023_merged_edited1.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_31_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_5_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_13_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_20_2024_lab_v17.csv"
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_20_2024_lab_v15.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = []
include_string_in_name_lst = [""]

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

accepted  DMPLS_MSCMR__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label2_uncertainty1_v15
accepted  DMPLS__coreset_fuse01_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label2_uncertainty1_v15
accepted  DMPLS__coreset_fuse05_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v15
accepted  DMPLS__coreset_fuse10_pos_loss1_wt035_use_slice_pos_uncertainty1_v15
accepted  DMPLS__coreset_fuse10_pos_wt01_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v15
accepted  DMPLS__coreset_fuse15_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label2_v15
accepted  DMPLS__coreset_pos_loss1_wt035_pos_loss2_wt010_use_phase_use_patient_v15
accepted  DMPLS__coreset_pos_loss1_wt035_use_slice_pos_v15
accepted  DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_bald200_v15
accepted  DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_la

In [12]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
mean_strings = round_strings[1:5]
#mean_strings = round_strings[0:4]
mean_string_col = "mean_test"
df = pd.DataFrame(index=index, 
                  columns=[mean_string_col] + sum(round_counts_strings, []))

In [13]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    mean_result_list = []
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results
            if round_string in mean_strings:
                mean_result_list.append(mean_result)
        sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)

/var/folders/j8/nh8m21z91210dgd4xzvdl9x80000gn/T/ipykernel_88750/2566401622.py:17: RuntimeWarning: invalid value encountered in double_scalars
  sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
df
df.sort_values(by=[mean_string_col], ascending=False)

,mean_test,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count
exp_name,,,,,,,,,,,,,,,,,,,,,
DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_v15,0.656152,0.351+/-0.080,5,0.560+/-0.058,5,0.583+/-0.064,5,0.726+/-0.012,5,0.755+/-0.024,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_pos_wt1000_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_v15,0.655243,0.313+/-0.051,5,0.563+/-0.066,5,0.592+/-0.086,5,0.709+/-0.030,5,0.757+/-0.022,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v15,0.653632,0.311+/-0.053,5,0.523+/-0.090,5,0.598+/-0.066,5,0.733+/-0.040,5,0.761+/-0.025,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_uncertainty15_v15,0.651273,0.315+/-0.085,4,0.551+/-0.058,4,0.562+/-0.040,4,0.737+/-0.012,4,0.755+/-0.015,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_uncertainty20_v15,0.646813,0.346+/-0.039,5,0.551+/-0.040,5,0.592+/-0.076,5,0.695+/-0.071,5,0.750+/-0.026,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_fuse15_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v15,0.645772,0.279+/-0.087,5,0.522+/-0.083,5,0.600+/-0.034,5,0.696+/-0.053,5,0.764+/-0.021,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_fuse01_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label6_uncertainty1_v15,0.64456,0.249+/-0.091,5,0.511+/-0.062,5,0.594+/-0.062,5,0.719+/-0.026,5,0.754+/-0.016,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_pos_wt1500_norm_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_label1_uncertainty25_v15,0.644051,0.290+/-0.071,4,0.545+/-0.056,4,0.616+/-0.051,4,0.699+/-0.033,4,0.716+/-0.018,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS__coreset_xt0_pos_loss1_wt033_pos_loss2_wt033_pos_loss3_wt033_use_slice_pos_use_phase_use_patient_v15,0.640765,0.281+/-0.067,5,0.479+/-0.084,5,0.603+/-0.080,5,0.726+/-0.027,5,0.755+/-0.020,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
